In [1]:
import warnings
warnings.filterwarnings('ignore')

# Matplotlib inline magic command
%matplotlib inline

In [2]:
# Initial imports
import pandas as pd
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
import numpy as np
import seaborn as sns
from collections import Counter
import psycopg2
from config import db_password
import json
from pandas.io.json import json_normalize
import mysql.connector
from sqlalchemy import create_engine
from os import walk
import time
import matplotlib as mpl
import matplotlib.pyplot as plt

In [3]:
# Scikit-Learn Model Selection and Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [50]:
# Scikit-Learn Model imports
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LinearRegression

In [5]:
# Sci-kit Learn Metrics imports
from sklearn.metrics import balanced_accuracy_score, roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import plot_confusion_matrix
from imblearn.metrics import classification_report_imbalanced

In [27]:
# Scikit-Learn Ensemble imports


In [ ]:
# Connect to AWS
def getDBengine1():
    user = 'postgres'
    passw = db_password
    host = 'policekillings.cv95lgysyvwq.us-east-1.rds.amazonaws.com'
    port = 5432
    database = 'PoliceKillings'
    engine_1 = create_engine(f'postgres://' + user + ':' + passw + '@' + host + ':' + str(port) + '/' + database ,
                             pool_recycle=3600, echo=False)

In [ ]:
# Load database into DataFrame
def getData1():
    s_statement = "SELECT * FROM PoliceKillings.police_killings"
    # df = pd.read_sql(s_statement, con=getDBengine())
    try:
        df = pd.read_sql(s_statement, con=getDBengine1())
    except:
        df = pd.DataFrame()
    return df

In [ ]:
df_data = getData1()
df_data.shape
print(df_data.columns)

In [7]:
# Load the data
file_path = 'Resources/2013-2020_Police_Killings_Revised.xlsx' 
killings_df = pd.read_excel(file_path)                         
killings_df

,Victim's age,Victim's gender,Victim's race,Date of Incident (month/day/year),County,Agency responsible for death,Cause of death,Symptoms of mental illness?,Armed/Unarmed Status,Alleged Threat Level (Source: WaPo),Fleeing (Source: WaPo),Body Camera (Source: WaPo),Geography,Encounter Type (DRAFT),Initial Reported Reason for Encounter (DRAFT),Call for Service? (DRAFT)
0,Unknown,Male,Unknown Race,2021-04-18,NaN,Detroit Police Department,Gunshot,NaN,Allegedly Armed,attack,NaN,NaN,NaN,Part 1 Violent Crime,shooting,NaN
1,Unknown,Male,Unknown Race,2021-04-18,NaN,Douglasville Police Department,Gunshot,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Unknown,Male,Unknown Race,2021-04-18,NaN,Fort Worth Police Department,Gunshot,NaN,Allegedly Armed,NaN,NaN,NaN,NaN,Other Non-Violent Offense,carjacking,NaN
3,Unknown,Male,White,2021-04-18,NaN,Burnsville Police Department,Gunshot,NaN,Allegedly Armed,attack,car,NaN,NaN,Other Non-Violent Offense,carjacking,NaN
4,Unknown,Male,Black,2021-04-17,NaN,Winter Haven Police Department,Gunshot,NaN,NaN,NaN,NaN,NaN,NaN,Domestic disturbance,Domestic disturbance (armed),NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9077,21,Male,White,2013-01-01,Douglas,Douglas County Sheriff's Office,Gunshot,Drug or alcohol use,Allegedly Armed,NaN,NaN,NaN,Rural,NaN,NaN,NaN
9078,26,Male,White,2013-01-01,Blue Earth,Mankato Department of Public Safety,Taser,No,Unarmed/Did Not Have Actual Weapon,NaN,NaN,NaN,Suburban,NaN,NaN,NaN
9079,21,Male,Hispanic,2013-01-01,Pueblo,Pueblo Police Department,Gunshot,No,Allegedly Armed,NaN,NaN,NaN,Suburban,NaN,NaN,NaN
9080,49,Male,Hispanic,2013-01-01,San Juan,Farmington Police Department,Gunshot,No,Allegedly Armed,NaN,NaN,NaN,Urban,NaN,NaN,NaN


In [8]:
# Rename columns
killings_df.rename(columns = {"Victim's age" : 'Victim_Age', "Victim's gender" : 'Victim_Gender', 
                              "Victim's race" : 'Victim_Race', 'Date of Incident (month/day/year)' : 'Date',
                              'Agency responsible for death' : 'Responsible_Agency', 'Cause of death' : 'Cause_of_Death',
                              'A brief description of the circumstances surrounding the death' : 'Brief_Description',
                              'Criminal Charges?' : 'Criminal_Charges', 'Symptoms of mental illness?' : 'Mental_Illness', 
                              'Armed/Unarmed Status' : 'Armed_Status', 'Alleged Threat Level (Source: WaPo)' : 'Threat_Level', 
                              'Fleeing (Source: WaPo)' : 'Fleeing', 'Body Camera (Source: WaPo)' : 'Body_Camera', 
                              'Encounter Type (DRAFT)' : 'Encounter_Type', 
                              'Initial Reported Reason for Encounter (DRAFT)' : 'Initial_Reason_for_Encounter',
                              'Call for Service? (DRAFT)' : 'Call_for_Service'}, inplace = True)

killings_df.head()

,Victim_Age,Victim_Gender,Victim_Race,Date,County,Responsible_Agency,Cause_of_Death,Mental_Illness,Armed_Status,Threat_Level,Fleeing,Body_Camera,Geography,Encounter_Type,Initial_Reason_for_Encounter,Call_for_Service
0,Unknown,Male,Unknown Race,2021-04-18,NaN,Detroit Police Department,Gunshot,NaN,Allegedly Armed,attack,NaN,NaN,NaN,Part 1 Violent Crime,shooting,NaN
1,Unknown,Male,Unknown Race,2021-04-18,NaN,Douglasville Police Department,Gunshot,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Unknown,Male,Unknown Race,2021-04-18,NaN,Fort Worth Police Department,Gunshot,NaN,Allegedly Armed,NaN,NaN,NaN,NaN,Other Non-Violent Offense,carjacking,NaN
3,Unknown,Male,White,2021-04-18,NaN,Burnsville Police Department,Gunshot,NaN,Allegedly Armed,attack,car,NaN,NaN,Other Non-Violent Offense,carjacking,NaN
4,Unknown,Male,Black,2021-04-17,NaN,Winter Haven Police Department,Gunshot,NaN,NaN,NaN,NaN,NaN,NaN,Domestic disturbance,Domestic disturbance (armed),NaN


In [9]:
# Delete Gender column
del killings_df['Victim_Gender']

In [10]:
# Find null values
for column in killings_df.columns:
    print(f'Column {column} has {killings_df[column].isnull().sum()} null values')

Column Victim_Age has 8 null values
Column Victim_Race has 0 null values
Column Date has 0 null values
Column County has 26 null values
Column Responsible_Agency has 4 null values
Column Cause_of_Death has 0 null values
Column Mental_Illness has 69 null values
Column Armed_Status has 6 null values
Column Threat_Level has 2394 null values
Column Fleeing has 3077 null values
Column Body_Camera has 3147 null values
Column Geography has 31 null values
Column Encounter_Type has 4310 null values
Column Initial_Reason_for_Encounter has 4310 null values
Column Call_for_Service has 5539 null values


In [11]:
# Replace empty/blank values for Age, Threat_Level, Fleeing, Body_Camera, Encounter_Type, Initial_Reason_for_Encounter, 
# Call_for_Service and re-verify null values

# Column: Age
killings_df['Victim_Age'].replace({'Unknown': '0'}, inplace = True) # Possible to fill in 'Unknown' with mean
                                                                    # of column?
# Column: Threat_Level
killings_df['Threat_Level'].replace({None: 'undetermined'}, inplace = True)

# Column: Fleeing
killings_df['Fleeing'].replace({None: 'unknown'}, inplace = True)

# Column: Body_Camera
killings_df['Body_Camera'].replace({None: 'unknown'}, inplace = True)

# Column: Encounter_Type
killings_df['Encounter_Type'].replace({None: 'Unknown'}, inplace = True)

# Column: Initial_Reason_for_Encounter
killings_df['Initial_Reason_for_Encounter'].replace({None: 'unknown'}, inplace = True)

# Column: Call_for_Service
killings_df['Call_for_Service'].replace({None: 'Unavailable'}, inplace = True)

# Verify null values
for column in killings_df.columns:
    print(f'Column {column} has {killings_df[column].isnull().sum()} null values')

Column Victim_Age has 8 null values
Column Victim_Race has 0 null values
Column Date has 0 null values
Column County has 26 null values
Column Responsible_Agency has 4 null values
Column Cause_of_Death has 0 null values
Column Mental_Illness has 69 null values
Column Armed_Status has 6 null values
Column Threat_Level has 0 null values
Column Fleeing has 0 null values
Column Body_Camera has 0 null values
Column Geography has 31 null values
Column Encounter_Type has 0 null values
Column Initial_Reason_for_Encounter has 0 null values
Column Call_for_Service has 0 null values


In [12]:
# Drop null values
killings_df = killings_df.dropna(how = 'any')
killings_df = killings_df.reset_index().drop(['index'], axis = 1)
killings_df

,Victim_Age,Victim_Race,Date,County,Responsible_Agency,Cause_of_Death,Mental_Illness,Armed_Status,Threat_Level,Fleeing,Body_Camera,Geography,Encounter_Type,Initial_Reason_for_Encounter,Call_for_Service
0,46,Hispanic,2021-04-14,Harris,Harris County Sheriff's Office,"Gunshot, Taser",Yes,Allegedly Armed,other,unknown,no,Suburban,Mental Health/Welfare Check,mental health crisis,Yes
1,48,Unknown Race,2021-04-10,Los Angeles,San Fernando Police Department,Gunshot,No,Allegedly Armed,attack,not fleeing,no,Urban,Part 1 Violent Crime,gunshots,Yes
2,0,Unknown Race,2021-04-10,Winnebago,Winnebago County Sheriff's Office,Gunshot,No,Unclear,undetermined,unknown,no,Suburban,Domestic disturbance,domestic disturbance,No
3,0,Unknown Race,2021-04-10,Jefferson,Jefferson Parish Sheriff's Office,Gunshot,No,Allegedly Armed,attack,unknown,no,Urban,Domestic disturbance,domestic disturbance,Yes
4,50,Unknown Race,2021-04-09,Clark,U.S. Marshals Service,Gunshot,No,Allegedly Armed,attack,not fleeing,no,Suburban,Other Crimes Against People,warrant (threatening judge),No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8969,21,White,2013-01-01,Douglas,Douglas County Sheriff's Office,Gunshot,Drug or alcohol use,Allegedly Armed,undetermined,unknown,unknown,Rural,Unknown,unknown,Unavailable
8970,26,White,2013-01-01,Blue Earth,Mankato Department of Public Safety,Taser,No,Unarmed/Did Not Have Actual Weapon,undetermined,unknown,unknown,Suburban,Unknown,unknown,Unavailable
8971,21,Hispanic,2013-01-01,Pueblo,Pueblo Police Department,Gunshot,No,Allegedly Armed,undetermined,unknown,unknown,Suburban,Unknown,unknown,Unavailable
8972,49,Hispanic,2013-01-01,San Juan,Farmington Police Department,Gunshot,No,Allegedly Armed,undetermined,unknown,unknown,Urban,Unknown,unknown,Unavailable


In [13]:
# Inspect column data types
killings_df.dtypes

Victim_Age                              object
Victim_Race                             object
Date                            datetime64[ns]
County                                  object
Responsible_Agency                      object
Cause_of_Death                          object
Mental_Illness                          object
Armed_Status                            object
Threat_Level                            object
Fleeing                                 object
Body_Camera                             object
Geography                               object
Encounter_Type                          object
Initial_Reason_for_Encounter            object
Call_for_Service                        object
dtype: object

In [14]:
# Change Victim_Age data type from object to int
killings_df['Victim_Age'] = killings_df['Victim_Age'].astype(str).astype(int)
killings_df.dtypes

Victim_Age                               int32
Victim_Race                             object
Date                            datetime64[ns]
County                                  object
Responsible_Agency                      object
Cause_of_Death                          object
Mental_Illness                          object
Armed_Status                            object
Threat_Level                            object
Fleeing                                 object
Body_Camera                             object
Geography                               object
Encounter_Type                          object
Initial_Reason_for_Encounter            object
Call_for_Service                        object
dtype: object

In [15]:
# Extract year from Date and create new column Year
killings_df['Year'] = pd.DatetimeIndex(killings_df['Date']).year

# Extract month from Date and create new column Month
killings_df['Month'] = pd.DatetimeIndex(killings_df['Date']).month

# Extract day from Date and create new column Day
killings_df['Day'] = pd.DatetimeIndex(killings_df['Date']).day

# Get day of week from Date and create new column Day_of_Week
killings_df['Day_of_Week'] = killings_df['Date'].dt.day_name()

In [16]:
# Get holiday from Date and create new column Holiday
dr = pd.date_range(start='2013-01-01', end='2021-04-18')
df = pd.DataFrame()
df['Date'] = dr

cal = calendar()
holidays = cal.holidays(start=dr.min(), end=dr.max())

killings_df['Holiday'] = killings_df['Date'].dt.date.astype('datetime64').isin(holidays)

In [ ]:
#killings_df

In [17]:
# Use get_dummies() to transform text values into numerical values
killings_encoded = pd.get_dummies(killings_df, columns = 
                                  ['Cause_of_Death', 'Mental_Illness', 'Armed_Status', 'Threat_Level', 'Fleeing', 
                                   'Body_Camera', 'Geography', 'County', 'Responsible_Agency', 'Encounter_Type', 
                                   'Initial_Reason_for_Encounter', 'Call_for_Service'])

killings_encoded.head()

,Victim_Age,Victim_Race,Date,Year,Month,Day,Day_of_Week,Holiday,Cause_of_Death_Asphyxiated,"Cause_of_Death_Baton, Pepper Spray, Physical Restraint",...,"Initial_Reason_for_Encounter_wellness check/erratic behavior, other: danger to self (suicide)","Initial_Reason_for_Encounter_wellness check/erratic behavior, shooting/shots fired","Initial_Reason_for_Encounter_wellness check/erratic behavior, warrant (failure to appear)","Initial_Reason_for_Encounter_wellness check/erratic behavior, weapons offense","Initial_Reason_for_Encounter_wellness check/erratic behavior, weapons offense (gun)",Call_for_Service_No,Call_for_Service_No,Call_for_Service_Unavailable,Call_for_Service_Yes,Call_for_Service_yes
0,46,Hispanic,2021-04-14,2021,4,14,Wednesday,False,0,0,...,0,0,0,0,0,0,0,0,1,0
1,48,Unknown Race,2021-04-10,2021,4,10,Saturday,False,0,0,...,0,0,0,0,0,0,0,0,1,0
2,0,Unknown Race,2021-04-10,2021,4,10,Saturday,False,0,0,...,0,0,0,0,0,1,0,0,0,0
3,0,Unknown Race,2021-04-10,2021,4,10,Saturday,False,0,0,...,0,0,0,0,0,0,0,0,1,0
4,50,Unknown Race,2021-04-09,2021,4,9,Friday,False,0,0,...,0,0,0,0,0,1,0,0,0,0


In [18]:
# Drop Date column
killings_df = killings_df.drop(['Date'], axis = 1)

In [19]:
killings_df['Victim_Race'].value_counts()

White               3935
Black               2259
Hispanic            1566
Unknown Race         903
Asian                134
Native American      127
Pacific Islander      50
Name: Victim_Race, dtype: int64

In [ ]:
#killings_df[killings_df['Victim_Race'] == 'Hispanic ']

In [20]:
killings_df[killings_df['Victim_Race'] != 'Hispanic ']
killings_df

,Victim_Age,Victim_Race,County,Responsible_Agency,Cause_of_Death,Mental_Illness,Armed_Status,Threat_Level,Fleeing,Body_Camera,Geography,Encounter_Type,Initial_Reason_for_Encounter,Call_for_Service,Year,Month,Day,Day_of_Week,Holiday
0,46,Hispanic,Harris,Harris County Sheriff's Office,"Gunshot, Taser",Yes,Allegedly Armed,other,unknown,no,Suburban,Mental Health/Welfare Check,mental health crisis,Yes,2021,4,14,Wednesday,False
1,48,Unknown Race,Los Angeles,San Fernando Police Department,Gunshot,No,Allegedly Armed,attack,not fleeing,no,Urban,Part 1 Violent Crime,gunshots,Yes,2021,4,10,Saturday,False
2,0,Unknown Race,Winnebago,Winnebago County Sheriff's Office,Gunshot,No,Unclear,undetermined,unknown,no,Suburban,Domestic disturbance,domestic disturbance,No,2021,4,10,Saturday,False
3,0,Unknown Race,Jefferson,Jefferson Parish Sheriff's Office,Gunshot,No,Allegedly Armed,attack,unknown,no,Urban,Domestic disturbance,domestic disturbance,Yes,2021,4,10,Saturday,False
4,50,Unknown Race,Clark,U.S. Marshals Service,Gunshot,No,Allegedly Armed,attack,not fleeing,no,Suburban,Other Crimes Against People,warrant (threatening judge),No,2021,4,9,Friday,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8969,21,White,Douglas,Douglas County Sheriff's Office,Gunshot,Drug or alcohol use,Allegedly Armed,undetermined,unknown,unknown,Rural,Unknown,unknown,Unavailable,2013,1,1,Tuesday,True
8970,26,White,Blue Earth,Mankato Department of Public Safety,Taser,No,Unarmed/Did Not Have Actual Weapon,undetermined,unknown,unknown,Suburban,Unknown,unknown,Unavailable,2013,1,1,Tuesday,True
8971,21,Hispanic,Pueblo,Pueblo Police Department,Gunshot,No,Allegedly Armed,undetermined,unknown,unknown,Suburban,Unknown,unknown,Unavailable,2013,1,1,Tuesday,True
8972,49,Hispanic,San Juan,Farmington Police Department,Gunshot,No,Allegedly Armed,undetermined,unknown,unknown,Urban,Unknown,unknown,Unavailable,2013,1,1,Tuesday,True


In [21]:
# Convert Victim_Race values to numbers
killings_df['Victim_Race'] = killings_df['Victim_Race'].replace('White', 1)
killings_df['Victim_Race'] = killings_df['Victim_Race'].replace('Black', 2)
killings_df['Victim_Race'] = killings_df['Victim_Race'].replace('Hispanic', 3)
killings_df['Victim_Race'] = killings_df['Victim_Race'].replace('Unknown Race', 4)
killings_df['Victim_Race'] = killings_df['Victim_Race'].replace('Asian', 5)
killings_df['Victim_Race'] = killings_df['Victim_Race'].replace('Native American', 6)
killings_df['Victim_Race'] = killings_df['Victim_Race'].replace('Pacific Islander', 7)

killings_df['Victim_Race']

0       3
1       4
2       4
3       4
4       4
       ..
8969    1
8970    1
8971    3
8972    3
8973    2
Name: Victim_Race, Length: 8974, dtype: int64

## Split Data into Training and Testing Sets

In [22]:
# Create features
X = pd.get_dummies(killings_df.drop(columns = ['Victim_Race']))

# Create target
y = pd.DataFrame(killings_df['Victim_Race'])

In [23]:
X.describe()

,Victim_Age,Year,Month,Day,County_Acadia,County_Acadiana,County_Ada,County_Adair,County_Adams,County_Aiken,...,Call_for_Service_Unavailable,Call_for_Service_Yes,Call_for_Service_yes,Day_of_Week_Friday,Day_of_Week_Monday,Day_of_Week_Saturday,Day_of_Week_Sunday,Day_of_Week_Thursday,Day_of_Week_Tuesday,Day_of_Week_Wednesday
count,8974.000000,8974.000000,8974.000000,8974.000000,8974.000000,8974.000000,8974.000000,8974.000000,8974.000000,8974.00000,...,8974.000000,8974.000000,8974.000000,8974.000000,8974.000000,8974.000000,8974.000000,8974.000000,8974.000000,8974.000000
mean,35.541899,2016.653109,6.298418,15.607421,0.000334,0.000111,0.001449,0.000223,0.005014,0.00078,...,0.636617,0.239692,0.000223,0.146757,0.135614,0.134834,0.137508,0.148540,0.149766,0.146980
std,14.557785,2.380105,3.458029,8.784326,0.018282,0.010556,0.038035,0.014928,0.070639,0.02792,...,0.481001,0.426920,0.014928,0.353884,0.342397,0.341565,0.344402,0.355654,0.356862,0.354106
min,0.000000,2013.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,26.000000,2015.000000,3.000000,8.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,34.000000,2017.000000,6.000000,16.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,45.000000,2019.000000,9.000000,23.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,107.000000,2021.000000,12.000000,31.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [24]:
# Check balance of target values
y['Victim_Race'].value_counts()

1    3935
2    2259
3    1566
4     903
5     134
6     127
7      50
Name: Victim_Race, dtype: int64

In [25]:
# Train, test, and split model
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    random_state = 78)

y_train.shape

(6730, 1)

In [ ]:
# ML model code
# Still need to pick a model.
# Test out all 3. 
# Most likely random forest is going to be the winning model because most of our data is discrete
# Random forest is better with discrete data

## Random Forest Model

In [28]:
# Create a random forest classifier
rf_model = RandomForestClassifier(n_estimators = 128, random_state = 78)
rf_model = rf_model.fit(X_train, y_train)

In [33]:
# Calculate Balanced Accuracy Score
rf_y_pred = rf_model.predict(X_test)
balanced_accuracy_score(y_test,rf_y_pred)

0.24827604352550053

In [34]:
y_test_np = np.array(y_test)
rf_y_pred_np = np.array(rf_y_pred)

In [35]:
y_test_np = y_test_np.reshape(-1, 1)
rf_y_pred_np = rf_y_pred_np.reshape(-1, 1)

In [46]:
# Calculate ROC accuracy score
roc_auc_score(y_test, rf_y_pred)

In [37]:
# Check for value types
rf_y_pred

array([1, 1, 1, ..., 2, 1, 2], dtype=int64)

In [38]:
# Display confusion matrix
rf_cm = confusion_matrix(y_test, rf_y_pred)
rf_cm

array([[841,  86,  24,   9,   1,   0,   1],
       [309, 251,  19,   7,   1,   0,   0],
       [237,  87,  66,   5,   0,   0,   0],
       [140,  35,  12,  27,   0,   0,   0],
       [ 30,   2,   3,   1,   0,   0,   0],
       [ 30,   2,   3,   1,   0,   0,   0],
       [  9,   3,   0,   0,   0,   0,   2]], dtype=int64)

In [39]:
# Print imbalanced classification report
print(classification_report_imbalanced(y_test, rf_y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          1       0.53      0.87      0.41      0.66      0.60      0.38       962
          2       0.54      0.43      0.87      0.48      0.61      0.36       587
          3       0.52      0.17      0.97      0.25      0.40      0.15       395
          4       0.54      0.13      0.99      0.20      0.35      0.11       214
          5       0.00      0.00      1.00      0.00      0.00      0.00        36
          6       0.00      0.00      1.00      0.00      0.00      0.00        36
          7       0.67      0.14      1.00      0.24      0.38      0.13        14

avg / total       0.51      0.53      0.71      0.47      0.52      0.29      2244



In [ ]:
# List feature sorted in descending order by feature importance - is this needed?
sorted(zip(rf_model.feature_importances_, X.columns), reverse = True)

In [40]:
# Model prediction
print(rf_y_pred)

[1 1 1 ... 2 1 2]


## Decision Tree